## Matriz de substituição de nucleotídeos: modelo K2P de Kimura, 1980.

### Calcula o scoring de substituição para match e mismatch entre as bases.

Similaridade = recompensa de +1

Transição = penalidade de -1

Transversão = penalidade de -2


```
   A  C  G  T
A  1 -2 -1 -2
C -2  1 -2 -1
G -1 -2  1 -2  
T -2 -1 -2  1  

```

Inserção ou deleção de base (gap) = penalidade de -3

In [23]:
# padronizar a sequência inserida pelo usuário (formato FASTA ou sequência livre de nucleotídeos).
# insere um * no início da sequência para construçã da matriz para o alinhamento.

def formata_sequencia(sequencia): 
    sequencia = sequencia.upper()
        
    if sequencia[0] == ">":
        sequencia = sequencia.splitlines()
        sequencia = sequencia[1:]
        sequencia = "".join(sequencia)
        sequencia = "*" + sequencia
        
    else:
        sequencia = sequencia.splitlines()
        sequencia = "".join(sequencia)
        sequencia = "*" + sequencia 
        
    return sequencia


In [ ]:
# verifica se a sequência é de DNA

def eh_dna(seq):
    if set(seq).issubset({"A", "C", "G", "T", "*"}):
        return True
    else:
        return False

In [24]:
# criar a matriz de substituição

def cria_matriz_subs():

    matriz = {"col": ["A", "C", "G", "T"],
                "A": [1, -2, -1, -2],
                "C": [-2, 1, -2, -1],
                "G": [-1, -2, 1, -2],
                "T": [-2, -1, -2, 1]}

    return matriz

In [25]:
# retorna o score do match ou mismatch entre 2 bases pareadas

def calcula_score(base1, base2, matriz_subs):

    j = matriz_subs["col"].index(base1)
    for base in "ACGT":
        if base2 == base:
            score = matriz_subs[base2][j]

            return score

In [26]:
# retorna o maior valor do L 

def valor_maximo(base1, base2, lado, cima, diagonal):
    # match
    if (base1 == base2) and (diagonal >= lado) and (diagonal >= cima):
        return diagonal
    # mismatch
    elif (base1 != base2) and (diagonal >= lado) and (diagonal >= cima):
        return diagonal
    # gap
    elif (lado > cima) and (lado > diagonal):
        return lado
    # gap
    else:
        return cima

In [27]:
# retorna de onde veio o maior valor do L

def cria_caminho(base1, base2, lado, cima, diagonal):
    # match
    if (base1 == base2) and (diagonal >= lado) and (diagonal >= cima):
        return "\\"
    # mismatch
    elif (base1 != base2) and (diagonal >= lado) and (diagonal >= cima):
        return "\\"
    # gap
    elif (lado > cima) and (lado > diagonal):
        return "-"
    # gap
    else:
        return "|"

In [38]:
# lcs = longest common subsequence
# montar 2 matrizes:
# pontuacao: receberá a pontuação (score) de cada par de base analisado
# caminho: recebe o caminho do qual a pontuação veio

def lcs(seq1, seq2, matriz_subs):

    pontuacao = []
    caminho = []
    g = -3

    # preencher a matriz
    for i in range(0, len(seq1)):
        pontuacao.append([0] * len(seq2))
        caminho.append([""] * len(seq2))

    # preencher a primeira coluna 
    for i in range(0, len(seq1)):
        pontuacao[i][0] = -3 * i
        caminho[i][0] = "|"
    # preenche a primeira linha
    for j in range(0, len(seq2)):
        pontuacao[0][j] = -3 * j
        caminho[0][j] = "-"

    # linha
    for i in range(1, len(seq1)):
        # coluna
        for j in range(1, len(seq2)):
        # devolver a valor maximo do L

            base1 = seq1[i]
            base2 = seq2[j] 

            # calcula o score (match e mismatch)
            s = calcula_score(base1, base2, matriz_subs)
            
            lado = pontuacao[i][j-1]
            cima = pontuacao[i-1][j]
            diagonal = pontuacao[i-1][j-1]
                                    
            pontuacao[i][j] = valor_maximo(base1, base2, lado + g, cima + g, diagonal + s)
            caminho[i][j] = cria_caminho(base1, base2, lado + g, cima + g, diagonal + s)

    return pontuacao, caminho

In [34]:
# imprime a matriz com a pontuação e o caminho

def imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho):

    print("\t", end="")

    for j in range(0, len(seq2)):
        print(seq2[j], end="\t")
    print()
    for i in range(0, len(seq1)):
        print(seq1[i], end="\t")

        for j in range(0, len(seq2)):
            print(matriz_pontuacao[i][j], matriz_caminho[i][j], end="\t", sep="")
        print()
    print()

In [41]:
# imprime o alinhamento ótimo entre as sequências

def gera_alinhamento(seq1, seq2, ponteiros, matriz_subs):
    
    ali_seq1 = ""
    ali_seq2 = ""
    
    match = 0
    mismatch = 0
    gap = 0
    score_final = 0
    g = -3

    # linhas
    i = len(seq1)-1
    # colunas
    j = len(seq2)-1

    while (i != 0) or (j != 0):

        s = calcula_score(seq1[i], seq2[j], matriz_subs)

        if matriz_caminho[i][j] == "\\" and seq1[i] == seq2[j]:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            match += 1
            score_final += s
            i -= 1
            j -= 1

        elif matriz_caminho[i][j] == "\\" and seq1[i] != seq2[j]:
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            mismatch += 1
            score_final += s
            i -= 1
            j -= 1    
    
        elif matriz_caminho[i][j] == "-":
            ali_seq1 = "-" + ali_seq1
            ali_seq2 = seq2[j] + ali_seq2
            gap += 1
            score_final += g
            j -= 1
    
        elif matriz_caminho[i][j] == "|":
            ali_seq1 = seq1[i] + ali_seq1
            ali_seq2 = "-" + ali_seq2
            gap += 1
            score_final += g
            i -= 1

    print(ali_seq1)  
    print(ali_seq2) 
    print()
    print(f"Matches = {match}")
    print(f"Mismatches = {mismatch}")
    print(f"Gaps = {gap}")
    print(f"Score final = {score_final}") 

In [36]:
sequencia1 = "AGTTCAGTTC"
sequencia2 = "AGTACAGGTTC"

seq1 = formata_sequencia(sequencia1)
seq2 = formata_sequencia(sequencia2)

In [42]:
matriz_subs = cria_matriz_subs()
matriz_pontuacao, matriz_caminho = lcs(seq1, seq2, matriz_subs)
gera_alinhamento(seq1, seq2, matriz_caminho, matriz_subs)

AGTTCA-GTTC
AGTACAGGTTC

Matches = 9
Mismatches = 1
Gaps = 1
Score final = 4


In [40]:
imprime_matriz(seq1, seq2, matriz_pontuacao, matriz_caminho)

	*	A	G	T	A	C	A	G	G	T	T	C	
*	0-	-3-	-6-	-9-	-12-	-15-	-18-	-21-	-24-	-27-	-30-	-33-	
A	-3|	1\	-2-	-5-	-8\	-11-	-14\	-17-	-20-	-23-	-26-	-29-	
G	-6|	-2|	2\	-1-	-4-	-7-	-10-	-13\	-16\	-19-	-22-	-25-	
T	-9|	-5|	-1|	3\	0-	-3-	-6-	-9-	-12-	-15\	-18\	-21-	
T	-12|	-8|	-4|	0\	1\	-1\	-4-	-7-	-10-	-11\	-14\	-17-	
C	-15|	-11|	-7|	-3|	-2\	2\	-1-	-4-	-7-	-10-	-12\	-13\	
A	-18|	-14\	-10|	-6|	-2\	-1|	3\	0-	-3-	-6-	-9-	-12-	
G	-21|	-17|	-13\	-9|	-5|	-4\	0|	4\	1\	-2-	-5-	-8-	
T	-24|	-20|	-16|	-12\	-8|	-6\	-3|	1|	2\	2\	-1\	-4-	
T	-27|	-23|	-19|	-15\	-11|	-9\	-6|	-2|	-1\	3\	3\	0-	
C	-30|	-26|	-22|	-18|	-14|	-10\	-9|	-5|	-4\	0|	2\	4\	

